In [1]:
import scanpy as sc
import numpy as np 
import pandas as pd
# openpyxl package to read excel# 'pip install openpyxl' if undownload it

In [2]:
supplementaryTable = "Supplementary table.xlsx"
dataset = "Placenta"
dataset_url = "https://cblast.gao-lab.org/Vento-Tormo_10x/Vento-Tormo_10x.h5"
CellType_ColumnName = 'cell_ontology_class'

In [3]:
# dataset download
from os.path import isfile
dataset_download_path = dataset_url.split('/')[-1]
if not isfile(dataset_download_path):
    !wget -O {dataset_download_path} {dataset_url}

--2022-02-28 18:12:55--  https://cblast.gao-lab.org/Vento-Tormo_10x/Vento-Tormo_10x.h5
Resolving cblast.gao-lab.org (cblast.gao-lab.org)... 159.138.49.219
Connecting to cblast.gao-lab.org (cblast.gao-lab.org)|159.138.49.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322868464 (308M) [application/octet-stream]
Saving to: ‘Vento-Tormo_10x.h5’

Vento-Tormo_10x.h5  100%[===================>] 307.91M  3.67MB/s    in 78s     

2022-02-28 18:14:13 (3.97 MB/s) - ‘Vento-Tormo_10x.h5’ saved [322868464/322868464]



In [4]:
def read_10x_h5(filename):
    import h5py
    import scanpy as sc
    from scipy import sparse
    import pandas as pd
    ds = h5py.File(filename, 'r')
    m=ds['exprs']
    def get_para(para):
        re = {}
        for k in ds[para].keys():
            re[k] = ds[para][k][:].astype(str)
        return re   
    ad=sc.AnnData(
      X = sparse.csr_matrix((m['data'][:],m['indices'][:],m['indptr'][:]),shape=m['shape']).tocsc(),
      var = pd.DataFrame(get_para('var'),index=ds['var_names'][:].astype(str)),
      obs = pd.DataFrame(get_para('obs'),index=ds['obs_names'][:].astype(str)),
      uns = get_para('uns')
    )
    return ad

In [21]:
adata = read_10x_h5(dataset_download_path)
adata.X = adata.X.tocsr()
df = pd.read_excel(supplementaryTable, sheet_name=dataset, na_filter=False)

In [22]:
def add_celltype(adata,celltype_columnname):
    adata.obs[celltype_columnname] = adata.obs[celltype_columnname].astype("category")
    Transferlist = [np.where(df['origin cell type']==CellTypeList)[0][0] for CellTypeList in adata.obs[celltype_columnname].values.categories]
    adata.obs['modelA id'] = df['modelA id'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
    adata.obs['modelC id'] = df['modelC id'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
    adata.obs['transfer cell type'] = df['transfer cell type'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
add_celltype(adata,CellType_ColumnName)

In [ ]:
adata.obs['origin cell type'] = adata.obs['cell_ontology_class']

In [23]:
adata.obs

,cell_ontology_class,cell_ontology_id,cell_type1,cluster,dataset_name,donor,latent_1,latent_10,latent_2,latent_3,...,organ,organism,platform,region,sample,tSNE1,tSNE2,modelA id,modelC id,transfer cell type
FCA7167222_TCTGGAACAGAGTGTG,stromal cell,CL:0000499,dS1,cluster_0,Vento-Tormo_10x,F20,1.0766369,-1.7757552,-0.45191085,0.25995594,...,Placenta,Homo sapiens,10x,Decidua,FCA7167222,1.7142705,-35.91711,-1,-1,unknown
FCA7167226_GGAACTTGTCTAAACC,stromal cell,CL:0000499,dS1,cluster_0,Vento-Tormo_10x,F20,0.98715854,-2.19444,-0.7691482,-0.62348914,...,Placenta,Homo sapiens,10x,Decidua,FCA7167226,-19.248367,-43.13908,-1,-1,unknown
FCA7196225_ACTATCTCATTCACTT,stromal cell,CL:0000499,dS1,cluster_0,Vento-Tormo_10x,F27,1.2470436,-2.0486107,-3.5434668,0.31293902,...,Placenta,Homo sapiens,10x,Decidua,FCA7196225,-24.78262,-17.058146,-1,-1,unknown
FCA7474063_AAAGCAAGTCAAACTC,stromal cell,CL:0000499,dS1,cluster_0,Vento-Tormo_10x,F36,0.5505987,-2.253791,-0.7096878,-0.09234756,...,Placenta,Homo sapiens,10x,Decidua,FCA7474063,-11.511428,-35.6276,-1,-1,unknown
FCA7196219_CTCGAAAGTTAAGTAG,stromal cell,CL:0000499,dS1,cluster_0,Vento-Tormo_10x,F25,1.020734,-2.0467753,-1.2073644,-0.93209827,...,Placenta,Homo sapiens,10x,Decidua,FCA7196219,-17.195986,-43.551888,-1,-1,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FCA7196219_GAGTCCGTCACGCGGT,glandular epithelial cell,CL:0000150,Epi2,cluster_50,Vento-Tormo_10x,F25,2.3324654,-1.9728192,-2.651536,0.96523887,...,Placenta,Homo sapiens,10x,Decidua,FCA7196219,-11.53461,51.73188,-1,-1,unknown
FCA7511882_AACCGCGCATTACCTT,glandular epithelial cell,CL:0000150,Epi2,cluster_50,Vento-Tormo_10x,F40,1.1974361,-1.0626452,-4.8679657,-0.3047195,...,Placenta,Homo sapiens,10x,Decidua,FCA7511882,-7.8867917,51.044357,-1,-1,unknown
FCA7167226_CTACCCATCCCACTTG,glandular epithelial cell,CL:0000150,Epi2,cluster_50,Vento-Tormo_10x,F20,1.4293659,-2.7362657,-1.5366335,-0.6728595,...,Placenta,Homo sapiens,10x,Decidua,FCA7167226,-11.806131,51.59371,-1,-1,unknown
FCA7167224_AGCCTAACACCATCCT,glandular epithelial cell,CL:0000150,Epi2,cluster_50,Vento-Tormo_10x,F19,1.0127151,-3.3082604,-0.94016093,-0.7704167,...,Placenta,Homo sapiens,10x,Decidua,FCA7167224,-12.203554,53.567444,-1,-1,unknown


In [28]:
adata.write_h5ad(f'../{dataset}.h5ad')

/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'cell_ontology_id' as categorical
/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'cell_type1' as categorical
/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removin

/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'transfer cell type' as categorical


In [29]:
!rm {dataset_download_path}